In [ ]:
import pandas as pd
from itertools import combinations

In [ ]:
def calculate_sample_standard_deviation(values):
    """Calculate the sample standard deviation for a list of values."""
    n = len(values)
    if n < 2:
        return None  # Standard deviation calculation requires at least two values
    mean_value = sum(values) / n
    standard_deviation = (sum((x - mean_value) ** 2 for x in values) / n) ** 0.5  # Square root of the variance
    return standard_deviation

def calculate_pairwise_rms_deviation(values):
    """Calculate RMS deviation based on pairwise squared differences."""
    n = len(values)
    if n < 2:
        return None  # Pairwise calculation requires at least two values
    if n == 2:
        # Special case: RMS for two values is the absolute difference
        return abs(values[0] - values[1])
    pairwise_differences = [(x - y) ** 2 for x, y in combinations(values, 2)]
    normalization_factor = n * (n - 1) / 2  # Total number of pairwise combinations
    rms_deviation = (sum(pairwise_differences) / normalization_factor) ** 0.5
    return rms_deviation

def calculate_pairwise_absolute_deviation(values):
    """Calculate absolute deviation based on pairwise absolute differences."""
    n = len(values)
    if n < 2:
        return None  # Pairwise calculation requires at least two values
    if n == 2:
        # Special case: Absolute deviation for two values is the absolute difference
        return abs(values[0] - values[1])
    pairwise_differences = [abs(x - y) for x, y in combinations(values, 2)]
    normalization_factor = n * (n - 1) / 2  # Total number of pairwise combinations
    absolute_deviation = sum(pairwise_differences) / normalization_factor
    return absolute_deviation

def process_triplets_with_deviations(file_path, num_annotators):
    """Process the input Excel file and calculate standard deviation, RMS deviation, and absolute deviation."""
    df = pd.read_excel(file_path, sheet_name=0)
    label_mapping = {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3}
    
    numeric_label_columns = []
    for i in range(1, num_annotators + 1):
        col = f"Answer.utilityValue.worker{i}"
        numeric_col = f"{col}_numeric"
        df[numeric_col] = df[col].map(label_mapping)
        numeric_label_columns.append(numeric_col)

    # Calculate deviations for each row
    stddevs = []
    rms_devs = []
    abs_devs = []
    deviation_details = []
    for _, row in df.iterrows():
        labels = [row[col] for col in numeric_label_columns if col in df.columns and not pd.isna(row[col])]
        stddev = calculate_sample_standard_deviation(labels)
        rms_dev = calculate_pairwise_rms_deviation(labels)
        abs_dev = calculate_pairwise_absolute_deviation(labels)
        if stddev is not None:
            stddev = round(stddev, 3)
        if rms_dev is not None:
            rms_dev = round(rms_dev, 3)
        if abs_dev is not None:
            abs_dev = round(abs_dev, 3)
        stddevs.append(stddev)
        rms_devs.append(rms_dev)
        abs_devs.append(abs_dev)
        deviation_details.append(f"Labels: {labels}, StdDev: {stddev}, RMS: {rms_dev}, Abs: {abs_dev}")
        
    df['Standard_Deviation'] = stddevs
    df['RMS_Deviation'] = rms_devs
    df['Absolute_Deviation'] = abs_devs
    df['Deviation_Calculation_Details'] = deviation_details
    
    overall_stddev = round(sum(filter(None, stddevs)) / len(stddevs) if stddevs else None, 3)
    overall_rms_dev = round(sum(filter(None, rms_devs)) / len(rms_devs) if rms_devs else None, 3)
    overall_abs_dev = round(sum(filter(None, abs_devs)) / len(abs_devs) if abs_devs else None, 3)
    overall_details = f"Overall StdDev: {overall_stddev}, Overall RMS: {overall_rms_dev}, Overall Abs: {overall_abs_dev}"
    
    output_file_path = file_path.replace('.xlsx', '_deviation_heuristics.xlsx')
    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Triplets_with_Deviations', index=False)
        overall_deviation_df = pd.DataFrame({
            'Metric': ['Overall StdDev', 'Overall RMS', 'Overall Abs'],
            'Value': [overall_stddev, overall_rms_dev, overall_abs_dev],
            'Details': [overall_details, overall_details, overall_details]
        })
        overall_deviation_df.to_excel(writer, sheet_name='Overall_Deviations', index=False)
    
    return output_file_path

In [ ]:
file_path = "/Users/innerpiece92/Desktop/NLP_Workspace/AArec/mturk/mturk-marketplace-ready/test/results_evaluation/restaurants/ground_truth_majority_vote.xlsx"
#file_path = "/Users/innerpiece92/Desktop/NLP_Workspace/AArec/mturk/mturk-marketplace-ready/test/results_evaluation/restaurants/human_ITA_agreement.xlsx"
output_file_path = process_triplets_with_deviations(file_path, num_annotators=3)
print(f"Output saved to: {output_file_path}")